# 9. Fourth model implementation

This notebook aims to implement the `ClassifierChain` with genetic algorithm just like described in the article `[d9e6797] LinearOrderingProblembasedClassifierChainusingGeneticAlgorithm.pdf`.

## 9.1. Recapping the implementation

After reading the article again, here's the main idea of the implementation:

* The order os labels is chosen based on the order that showed a better "fitness score". Choosing a better order is handled by the genetic algorithm, and we don't have to worry about (apart from ensuring that the parameters are the same that the article used).
* The fitness score is where we get the "linear ordering problem" (LOP). Here's how it works.
  * Take a label order, for example, `[1, 4, 2, 3]`. It has `n=4` labels.
  * Build a `n x n` matrix. Rows index are `[1, 4, 2, 3]`, and the columns are `[1, 4, 2, 3]`.
  * Each cell of this matrix will have a value that is equal to the **conditional entropy** of the label in the row, given the label in the column. For example, the cell `[1, 4]` will have the value of the conditional entropy of the label `1`, given the label `4`.
  * Now sum the upper triangle of this matrix. That's the **fitness score**.

## 9.2. Setup

In [45]:
from skmultilearn.dataset import load_dataset
import numpy as np
from skmultilearn.problem_transform import ClassifierChain
import pygad
from typing import List
import sklearn.metrics as metrics
from typing import Any, Optional
import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import math
from numpy.typing import NDArray
from typing import Dict

In [2]:
desired_datasets = ["scene", "emotions", "birds"]

datasets = {}
for dataset_name in desired_datasets:
    print(f"getting dataset `{dataset_name}`")
    
    full_dataset = load_dataset(dataset_name, "undivided")
    X, y, _, _ = full_dataset

    train_dataset = load_dataset(dataset_name, "train")
    X_train, y_train, _, _ = train_dataset

    test_dataset = load_dataset(dataset_name, "test")
    X_test, y_test, _, _ = test_dataset

    datasets[dataset_name] = {
        "X": X,
        "y": y,
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test,
        "y_test": y_test,
        "rows": X.shape[0],
        "labels_count": y.shape[1]
    }

for name, info in datasets.items():
    print("===")
    print(f"information for dataset `{name}`")
    print(f"rows: {info['rows']}, labels: {info['labels_count']}")


getting dataset `scene`
scene:undivided - exists, not redownloading
scene:train - exists, not redownloading
scene:test - exists, not redownloading
getting dataset `emotions`
emotions:undivided - exists, not redownloading
emotions:train - exists, not redownloading
emotions:test - exists, not redownloading
getting dataset `birds`
birds:undivided - exists, not redownloading
birds:train - exists, not redownloading
birds:test - exists, not redownloading
===
information for dataset `scene`
rows: 2407, labels: 6
===
information for dataset `emotions`
rows: 593, labels: 6
===
information for dataset `birds`
rows: 645, labels: 19


## 9.3. Playing around with entropy calculation

In [22]:
y = datasets["scene"]["y_train"]
y.todense()

matrix([[1, 0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1]], dtype=int64)

In [4]:
y.shape

(1211, 6)

In [5]:
label_count = y.shape[1]
rows_count = y.shape[0]

probs = []

for label in range(label_count):
    instances_with_label = y[:, label].todense().sum()
    probs.append(instances_with_label / rows_count)

probs

[0.1874483897605285,
 0.13625103220478943,
 0.16267547481420314,
 0.16184971098265896,
 0.22873658133773742,
 0.18497109826589594]

In [6]:
entropies = []

for prob in probs:
    entropy = -1 * prob * math.log(prob, 2)
    entropies.append(entropy)

entropies

[0.45276933803928565,
 0.3918117707258502,
 0.4261985731270658,
 0.42522342518043577,
 0.4868065668618529,
 0.45033585713511676]

In [7]:
joint_entropies = []

for label in range(label_count):
    joint_entropies.append([])

    for j in range(label_count):
        and_prob = probs[label] * probs[j]
        joint_entropy = -1 * and_prob * math.log(and_prob, 2)
        joint_entropies[label].append(joint_entropy)

joint_entropies

[[0.169741766696809,
  0.13513477517031391,
  0.15354470329775657,
  0.15298803284199744,
  0.19481601760076195,
  0.1681639729896544],
 [0.13513477517031391,
  0.10676951638276679,
  0.12180816135339254,
  0.12135175245007311,
  0.15594958218271368,
  0.13383257891815417],
 [0.15354470329775657,
  0.12180816135339254,
  0.1386641104971626,
  0.1381535384751864,
  0.17667869399503078,
  0.1520930175359874],
 [0.15298803284199744,
  0.12135175245007311,
  0.1381535384751864,
  0.13764457693701967,
  0.17605365473154738,
  0.15154077228645788],
 [0.19481601760076195,
  0.15594958218271368,
  0.17667869399503078,
  0.17605365473154738,
  0.22270093975348185,
  0.19305342973037357],
 [0.1681639729896544,
  0.13383257891815417,
  0.1520930175359874,
  0.15154077228645788,
  0.19305342973037357,
  0.16659823616559233]]

In [8]:
def cond_entropy(x, y):
    return joint_entropies[x][y] - entropies[y]

cond_entropy(4,2)

-0.24951987913203505

Okay, I managed to implement the calculation. But the article gives the impression that, when `x` and `y` are the same, the conditional entropy should be zero. But that's not what I'm getting.

In [9]:
cond_entropy(0,0)

-0.2830275713424767

In [10]:
cond_entropy(1,1)

-0.28504225434308345

In [11]:
cond_entropy(2,2)

-0.28753446262990323

**I think that I understood what is wrong**. We are supposed to sum the probabilities, as the probability are calculated for each value that the variable may take. In the case of the labels, since they are binary, the values are either 0 or 1. We have to calculate the probabilities for 0, then 1, then sum them.

In [12]:
label_count = y.shape[1]
rows_count = y.shape[0]

probs = []

for label in range(label_count):
    instances_with_label = y[:, label].todense().sum() # value = 1
    instances_without_label = rows_count - instances_with_label # value = 0

    probs.append([
        instances_without_label / rows_count,  # value = 0
        instances_with_label / rows_count,    # value = 1
    ])

probs

[[0.8125516102394715, 0.1874483897605285],
 [0.8637489677952106, 0.13625103220478943],
 [0.8373245251857968, 0.16267547481420314],
 [0.838150289017341, 0.16184971098265896],
 [0.7712634186622626, 0.22873658133773742],
 [0.815028901734104, 0.18497109826589594]]

In [13]:
entropies = []

for prob in probs:
    results = []
    for value in [0,1]:
        prob_for_value = prob[value]
        summand = prob_for_value * math.log(prob_for_value, 2)
        results.append(summand)
    
    entropy = -1 * sum(results)
    entropies.append(entropy)

entropies

[0.6961030672262447,
 0.5743357592196299,
 0.6406718924240619,
 0.6387163439963871,
 0.7758023710944532,
 0.690832038687419]

In [14]:
joint_entropies = []

# for prob in probs:
#     for value_i in [0,1]:
#         for value_j in [0,1]:
#             and_prob = prob[value_i] * prob[value_j]

#             joint_entropy = -1 * and_prob * math.log(and_prob, 2)
#             joint_entropies.append(joint_entropy)

for x in range(label_count):
    joint_entropies.append([])

    for y in range(label_count):
        results = []
        for value_i in [0,1]:
            for value_j in [0,1]:
                and_prob = probs[x][value_i] * probs[y][value_j]
                summand = and_prob * math.log(and_prob, 2)
                results.append(summand)
                print(f"for x={x}, y={y}, value_i={value_i}, value_j={value_j}, and_prob={and_prob}, math.log(and_prob, 2) = {math.log(and_prob, 2)}, summand={summand}")

        joint_entropy = -1 * sum(results)
        joint_entropies[x].append(joint_entropy)

joint_entropies

for x=0, y=0, value_i=0, value_j=0, and_prob=0.660240119302758, math.log(and_prob, 2) = -0.5989372887101777, summand=-0.3954424269528782
for x=0, y=0, value_i=0, value_j=1, and_prob=0.1523114909367135, math.log(and_prob, 2) = -2.714903306758502, summand=-0.4135109704014011
for x=0, y=0, value_i=1, value_j=0, and_prob=0.1523114909367135, math.log(and_prob, 2) = -2.714903306758502, summand=-0.4135109704014011
for x=0, y=0, value_i=1, value_j=1, and_prob=0.035136898823815, math.log(and_prob, 2) = -4.8308693248068275, summand=-0.169741766696809
for x=0, y=1, value_i=0, value_j=0, and_prob=0.7018406146246798, math.log(and_prob, 2) = -0.5107846578024762, summand=-0.35848941817294666
for x=0, y=1, value_i=0, value_j=1, and_prob=0.11071099561479174, math.log(and_prob, 2) = -3.175129579803602, summand=-0.3515217569860321
for x=0, y=1, value_i=1, value_j=0, and_prob=0.16190835317053082, math.log(and_prob, 2) = -2.626750675850801, summand=-0.425292876116582
for x=0, y=1, value_i=1, value_j=1, and

[[1.3922061344524894,
  1.2704388264458748,
  1.3367749596503065,
  1.3348194112226317,
  1.471905438320698,
  1.3869351059136636],
 [1.2704388264458748,
  1.14867151843926,
  1.2150076516436918,
  1.2130521032160169,
  1.350138130314083,
  1.265167797907049],
 [1.3367749596503065,
  1.2150076516436918,
  1.2813437848481235,
  1.279388236420449,
  1.416474263518515,
  1.3315039311114807],
 [1.3348194112226317,
  1.2130521032160169,
  1.279388236420449,
  1.2774326879927738,
  1.4145187150908403,
  1.3295483826838062],
 [1.471905438320698,
  1.350138130314083,
  1.4164742635185146,
  1.4145187150908403,
  1.5516047421889063,
  1.4666344097818722],
 [1.3869351059136636,
  1.265167797907049,
  1.3315039311114807,
  1.329548382683806,
  1.4666344097818722,
  1.3816640773748379]]

In [15]:
import numpy as np

def get_joint_entropy(x, y, probs):
    results = []
    for value_i in [0,1]:
        for value_j in [0,1]:
            and_prob = probs[x][value_i] * probs[y][value_j]
            # summand = and_prob * np.log2(and_prob) 

            if and_prob > 0:  # Avoid taking the log of 0
                summand = and_prob * np.log2(and_prob)
                results.append(summand)
    
    return -1 * sum(results)

get_joint_entropy(0, 0, probs)

1.3922061344524894

In [16]:
def cond_entropy(x, y):
    return joint_entropies[x][y] - entropies[y]

In [17]:
print(joint_entropies[1][1])
print(entropies[1])
print(entropies[1])
print(entropies[1]+ entropies[1])


1.14867151843926
0.5743357592196299
0.5743357592196299
1.1486715184392597


In [18]:
for label in range(label_count):
    for j in range(label_count):
        print(cond_entropy(label,j))

0.6961030672262447
0.6961030672262449
0.6961030672262446
0.6961030672262446
0.6961030672262448
0.6961030672262446
0.5743357592196301
0.5743357592196301
0.57433575921963
0.5743357592196298
0.5743357592196298
0.57433575921963
0.6406718924240618
0.640671892424062
0.6406718924240616
0.6406718924240619
0.6406718924240619
0.6406718924240616
0.638716343996387
0.638716343996387
0.6387163439963871
0.6387163439963867
0.6387163439963871
0.6387163439963871
0.7758023710944533
0.775802371094453
0.7758023710944527
0.7758023710944532
0.7758023710944532
0.7758023710944532
0.6908320386874189
0.6908320386874192
0.6908320386874188
0.6908320386874188
0.690832038687419
0.6908320386874188


In [19]:
print(cond_entropy(1,3))
print(cond_entropy(3,1))

0.5743357592196298
0.638716343996387


Honestly, I think that I got it right... I was expecting conditional_entropy to be zero, as the article states that:

> If condEntropy(X /Y ) = 0, X has no uncertainty in the Y ’s presence, i.e., X can be determined entirely by Y.

So if X and Y are the same variable, obviously knowing X determines Y. But the calculations provided in the article, which I could verify [here](https://www.ece.tufts.edu/ee/194NIT/lect01.pdf), [here](https://en.wikipedia.org/wiki/Joint_entropy) and even [here](https://gist.github.com/kudkudak/dabbed1af234c8e3868e) and also [here](https://www.cs.cmu.edu/~venkatg/teaching/ITCS-spr2013/notes/lect-jan17.pdf), will obviously lead to the joint entropy of X,X always being the sum of the entropy of X and the entropy of X. And since conditional entropy is the joint entropy minus the entropy of Y, it will always be the entropy of X.

The reason for that is that the formula for the joint entropy takes two sums, one for X and one for Y. So when X and Y are the same, we have to sum the entropy of X twice. Therefore, the joint entropy of X,X will always be the entropy of X multiplied by 2. If you subtract the entropy of Y, which is zero, you will always get the entropy of X.

Other implementation in Python seen [here](https://datascience.stackexchange.com/questions/58565/conditional-entropy-calculation-in-python-hyx).

_So my implementation is most likely correct_. The only weird thing is that the final conditional entropy values are very similar across the same row (same X, or same Y). I don't know if that's normal or not.

## 9.4. Stablish code for entropy calculation

Now that we are comfortable with the entropy calculation, let's make a more "stable" code for it.

In [20]:
# by Copilot:

class Entropy:
    def __init__(self, probs):
        self.probs = probs
        self.label_count = len(probs)
        self.entropies = self._get_entropies()
        self.joint_entropies = self._get_joint_entropies()
    
    def _get_entropies(self):
        entropies = []

        for prob in self.probs:
            results = []
            for value in [0,1]:
                prob_for_value = prob[value]
                summand = prob_for_value * math.log(prob_for_value, 2)
                results.append(summand)
            
            entropy = -1 * sum(results)
            entropies.append(entropy)

        return entropies

    def _get_joint_entropies(self):
        joint_entropies = []

        for x in range(self.label_count):
            joint_entropies.append([])

            for y in range(self.label_count):
                results = []
                for value_i in [0,1]:
                    for value_j in [0,1]:
                        and_prob = self.probs[x][value_i] * self.probs[y][value_j]
                        summand = and_prob * math.log(and_prob, 2)
                        results.append(summand)
                        # print(f"for x={x}, y={y}, value_i={value_i}, value_j={value_j}, and_prob={and_prob}, math.log(and_prob, 2) = {math.log(and_prob, 2)}, summand={summand}")

                joint_entropy = -1 * sum(results)
                joint_entropies[x].append(joint_entropy)

        return joint_entropies

    def get_cond_entropy(self, x, y):
        return self.joint_entropies[x][y] - self.entropies[y]

    def get_cond_entropy_matrix(self):
        matrix = []

        for i in range(self.label_count):
            matrix.append([])
            for j in range(self.label_count):
                matrix[i].append(self.get_cond_entropy(i,j))
        
        return matrix

    def get_cond_entropy_matrix(self):
        matrix = []

        for i in range(self.label_count):
            matrix.append

In [47]:
Probabilities = Dict[int, Dict[int, float]]

def calculate_probabilities(y: NDArray[np.int64]) -> Probabilities:
    dense_y = y.todense()

    label_count = dense_y.shape[1]
    rows_count = dense_y.shape[0]

    probs = {}

    for label in range(label_count):
        probs[label] = {}
        y_label_specific = np.asarray(dense_y[:, label]).reshape(-1)
        # convert_matrix_to_vector

        possible_values = np.unique(y_label_specific)

        for value in possible_values:
            instances_with_label = np.count_nonzero(y_label_specific == value)
            probs[label][value] = instances_with_label / rows_count
    
    return probs

y = datasets["scene"]["y_train"]
probs = calculate_probabilities(y)
probs

{0: {0: 0.8125516102394715, 1: 0.1874483897605285},
 1: {0: 0.8637489677952106, 1: 0.13625103220478943},
 2: {0: 0.8373245251857968, 1: 0.16267547481420314},
 3: {0: 0.838150289017341, 1: 0.16184971098265896},
 4: {0: 0.7712634186622626, 1: 0.22873658133773742},
 5: {0: 0.815028901734104, 1: 0.18497109826589594}}

In [48]:
def calculate_entropies(probabilities: Probabilities) -> Dict[int, float]:
    entropies = {}

    for label, calculated_probabilities in probabilities.items():
        results = []
        for _, prob in calculated_probabilities.items():
            summand = prob * math.log(prob, 2)
            results.append(summand)
        
        entropy = -1 * sum(results)
        entropies[label] = entropy

    return entropies

calculate_entropies(probs)

{0: 0.6961030672262447,
 1: 0.5743357592196299,
 2: 0.6406718924240619,
 3: 0.6387163439963871,
 4: 0.7758023710944532,
 5: 0.690832038687419}